# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# CREATING A CLUSTER
from cassandra.cluster import Cluster
cluster = Cluster()

# CREATING THE SESSION
session = cluster.connect()

#### Create Keyspace

In [11]:
# SESSION EXECUTION
session.execute("""
      CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")

#### Set Keyspace

In [12]:
# SETTING THE KEYSPACE
session.set_keyspace("mykeyspace")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [95]:
# HERE WE ARE CREATING THE MUSIC_HISTORY_FINALVER 
# WE HAVE SESSION_ID & ITEM_IN_SESSION AS THE PRIMARY KEYS BECAUSE WE WANT TO FILTER OUT TO SESSION_ID & ITEM_IN_SESSION 
table1 = """CREATE TABLE IF NOT EXISTS music_history_finalver1 (
session_id int, item_in_session int, artist text, song_title text, song_length float, 
PRIMARY KEY(session_id, item_in_session))"""         

# TABLE1 EXEUCTION
session.execute(table1)

In [96]:
file = 'event_datafile_new.csv'

# WE ARE LOOPING THROUGH THE CSV
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # WE ARE INSERTING THE DATA FROM THE CSV INTO THE MUSIC_HISTORY_FINALVER TABLE
        query = "INSERT INTO music_history_finalver1 (session_id, item_in_session, artist, song_title, song_length) VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[10], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [97]:
# THE QUERY WE WANNT TO EXECUTE 
verification = "SELECT artist, song_title,song_length FROM music_history_finalver1 WHERE session_id = 338 and item_in_session = 4" 

# CREATING LISTS TO CREATE A DATAFRAME FOR THE RESULTS LATER ON
artist = []
song_title = []
song_length = [] 

# EXECUTING THE RESULTS AND APPENDING THE DATA INTO THE CORRECT LISTS ABOVE
try:
    results = session.execute(verification)
    for row in results:
        artist.append(row[0])
        song_title.append(row[1])
        song_length.append(row[2])
except Exception as error:
    print(error)

# CREATING A DATAFRAME FOR THE RESULTS 
try: 
    df = pd.DataFrame({
        "artist":artist,
        "song_title":song_title,
        "song_length": song_length
    })
    print(df)
    #print(df(index=False)) -> WE CAN RUN THE RESULTS WITHOUT SEEING THE INDEX IF REQUIRED WHICH WILL CONVER THE DISPLAYED RESULTS INTO A STRING
except Exception as error:
    print(error)

      artist song_title  song_length
0  Faithless         50   495.307312


In [91]:
# WE ARE CREATING TABLE 2 (USER_INFORMATION_FINALVER)
# WE ARE USING USER_ID AS THE PARTITION KEY WITH SESSION_ID AND ITEM_IN_SESSION BEING THE CLUSTERING COLUMNS TO CREATE THE UNIQUE PRIMARY KEY

table2 = """CREATE TABLE IF NOT EXISTS user_information_finalver (user_id int, session_id int, item_in_session int, artist text, song text, user_first_name text, user_last_name text,
PRIMARY KEY((user_id, session_id), item_in_session)
"""      
# TABLE2 EXECUTION
session.execute(table2)

file = 'event_datafile_new.csv'

# LOOPING THROUGH THE CSV
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # WE ARE INSERTING THE DATA FROM THE CSV INTO THE USER_INFORMATION_FINALVER TABLE
        query = "INSERT INTO user_information_finalver (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# QUERY WE WANT TO EXECUTE
verification2 = "SELECT artist, song, user_first_name, user_last_name FROM user_information_finalver WHERE user_id = 10 and session_id = 182"

# CREATING LISTS TO CREATE A DATAFRAME FOR THE RESULTS LATER ON
artist = []
song = []
user_first_name = []
user_last_name = []
try:
    # EXECUTING THE RESULTS AND APPENDING THE DATA INTO THE CORRECT LISTS ABOVE
    results = session.execute(verification2)
    for rows in results:
        artist.append(row[0])
        song.append(rows[1]) 
        user_first_name.append(rows[2])
        user_last_name.append(rows[3])
except Exception as error:
    print(error)

# CREATING A DATAFRAME FOR THE RESULTS
try: 
    df = pd.DataFrame({
    "artist":artist,
    "song" : song, 
    "user_first_name":user_first_name,
    "user_last_name": user_last_name
    })
    print(df)
    #print(df(index=False)) -> WE CAN RUN THE RESULTS WITHOUT SEEING THE INDEX IF REQUIRED WHICH WILL CONVER THE DISPLAYED RESULTS INTO A STRING

except Exception as error: 
    print(error)

      artist                                               song  \
0  Faithless                                 Keep On Keepin' On   
1  Faithless                                        Greece 2000   
2  Faithless                                          Kilometer   
3  Faithless  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  user_first_name user_last_name  
0          Sylvie           Cruz  
1          Sylvie           Cruz  
2          Sylvie           Cruz  
3          Sylvie           Cruz  


In [88]:
# CREATING THE LISTENING_HISTORY_FINALVER TABLE 
# THE PRIMARY KEYS ARE SONG AND USER_ID. SONG COMES FIRST BECAUSE WE WANT TO FILTER OUT TO SONGS AND USER ID IS THE CLUSTERING COLUMN TO ORDER THE DATA WITHIN EACH SONG PARTITION

table3 = """CREATE TABLE IF NOT EXISTS listening_history_finalver (song text, user_id int, user_first_name text, user_last_name text,
PRIMARY KEY(song, user_id))
"""      

# TABLE3 EXECUTION
session.execute(table3)

file = 'event_datafile_new.csv'

# LOOPING THROUGH THE CSV
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listening_history_finalver (song, user_id, user_first_name, user_last_name) VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# QUERY WE WANT TO EXECUTE
verification3 = "SELECT user_first_name, user_last_name FROM listening_history_finalver WHERE song = 'All Hands Against His Own'"

# CREATING LISTS TO CREATE A DATAFRAME FOR THE RESULTS LATER ON
user_first_name = []
user_last_name = []

try:
    # EXECUTING THE RESULTS AND APPENDING THE DATA INTO THE CORRECT LISTS ABOVE
    results = session.execute(verification3)
    for rows in results:
        user_first_name.append(rows[0])
        user_last_name.append(rows[1])
except Exception as error:
    print(error)
    
# CREATING A DATAFRAME FOR THE RESULTS 
try:
    df = pd.DataFrame({
        "user_first_name":user_first_name,
        "user_last_name":user_last_name
    })
    print(df)
    #print(df(index=False)) -> WE CAN RUN THE RESULTS WITHOUT SEEING THE INDEX IF REQUIRED WHICH WILL CONVER THE DISPLAYED RESULTS INTO A STRING

except Exception as error:
    print(error)

  user_first_name user_last_name
0      Jacqueline          Lynch
1           Tegan         Levine
2            Sara        Johnson


### Drop the tables before closing out the sessions

In [98]:
# DROPPING ALL THE TABLES
session.execute("DROP TABLE music_history_finalver1;")
session.execute("DROP TABLE user_information_finalver;")
session.execute("DROP TABLE listening_history_finalver;")

### Close the session and cluster connection¶

In [99]:
# CLOSING THE SESSION & CLUSTER
session.shutdown()
cluster.shutdown()